In [ ]:
import os
import json
import math
import clingo
from itertools import batched
import matplotlib.pyplot as plt

SCHEDULER_SOURCE = './Source/schedule_discrete.lp'
DATA_DIR = './Data'

# Context class for ASP
class Context:
    def clamp(self, x, lower, upper): return min(max(x, lower), upper)
    def max(self, a, b): return max(a, b)
    def min(self, a, b): return min(a, b)

def run_solver(data_file):
    """Run Clingo solver for one data file and return results lists."""
    with open(data_file) as f:
        data = json.load(f)

    with open(SCHEDULER_SOURCE) as f:
        scheduler_program = f.read()

    # Build input program
    program_input = ''
    batch_count = 4
    for (hour, slots) in enumerate(batched(data['schedule_input'], n=batch_count)):
        slot_count = len(slots)

        # Handle None values safely
        price = sum(((slot.get('price_buying') or 0) * 1000 for slot in slots)) / slot_count
        production_ac = sum((slot.get('production_forecast_ac') or 0 for slot in slots))
        production_dc = sum((slot.get('production_forecast_dc') or 0 for slot in slots))
        consumption = sum((slot.get('consumption_forecast') or 0 for slot in slots))

        program_input += f'price({hour + 1}, {round(price)}).\n'
        program_input += f'production({hour + 1}, {round(production_ac + production_dc)}).\n'
        program_input += f'consumption({hour + 1}, {round(consumption)}).\n'

    hours = math.ceil(len(data['schedule_input']) / batch_count)
    site = data['site_info']
    constants = {
        'hours': hours,
        'max_charge_rate': site['max_charge_amount'] * batch_count * 100 / site['battery_capacity'],
        'max_discharge_rate': site['max_discharge_amount'] * batch_count * 100 / site['battery_capacity'],
        'charge_efficiency': site['charge_efficiency_from_ac'] * 100,
        'discharge_efficiency': site['discharge_efficiency_to_ac'] * 100,
    }

    parameters = ['--stats', '--parallel-mode', '4'] + \
                 [s for (key, value) in constants.items() for s in ['-c', f'{key}={round(value)}']]

    ctl = clingo.Control(parameters)
    ctl.add("input", [], program_input)
    ctl.add("base", [], scheduler_program)
    ctl.ground([("input", []), ("base", [])], context=Context())

    results = {"schedule": {}, "soc": {}, "grid_usage": {}, "price": {}, "cost": {}}

    def collect_model(model):
        atoms = model.symbols(shown=True)
        for atom in atoms:
            name = atom.name
            args = atom.arguments
            if name in results and len(args) == 2:
                h = int(str(args[0]))
                v = str(args[1])
                if v.lstrip('-').isdigit():
                    v = int(v)
                results[name][h] = v

    ctl.solve(on_last=collect_model)

    # Convert results to lists
    hours = sorted(results["soc"].keys())
    hours = [h for h in hours if h > 0]
    soc = [int(results["soc"][h]) for h in hours]
    grid_usage = [int(results["grid_usage"].get(h, 0)) for h in hours]
    price = [int(results["price"].get(h, 0)) for h in hours]
    cost = [int(results["cost"].get(h, 0)) for h in hours]

    return soc, grid_usage, price, cost

# --- Loop over all files ---
all_soc, all_grid, all_price, all_cost = [], [], [], []
per_day_avgs = {}

for file in os.listdir(DATA_DIR):
    if file.startswith("daily_") and file.endswith(".json"):
        path = os.path.join(DATA_DIR, file)
        soc, grid, price, cost = run_solver(path)
        all_soc.extend(soc)
        all_grid.extend(grid)
        all_price.extend(price)
        all_cost.extend(cost)
        # Store per-day averages
        if soc:  # avoid empty
            per_day_avgs[file] = {
                "soc": sum(soc)/len(soc),
                "grid": sum(grid)/len(grid),
                "price": sum(price)/len(price),
                "cost": sum(cost)/len(cost)
            }

# --- Compute global averages ---
avg_soc = sum(all_soc) / len(all_soc) if all_soc else 0
avg_grid = sum(all_grid) / len(all_grid) if all_grid else 0
avg_price = sum(all_price) / len(all_price) if all_price else 0
avg_cost = sum(all_cost) / len(all_cost) if all_cost else 0

print("=== Global Averages across all files ===")
print(f"Average SoC: {avg_soc:.2f}")
print(f"Average Grid Usage: {avg_grid:.2f}")
print(f"Average Price: {avg_price:.2f}")
print(f"Average Cost: {avg_cost:.2f}")

# --- Plot global averages ---
labels = ["SoC", "Grid Usage", "Price", "Cost"]
averages = [avg_soc, avg_grid, avg_price, avg_cost]

plt.figure(figsize=(8,6))
plt.bar(labels, averages, color=["blue","orange","green","red"])
plt.title("Global Average Metrics Across All Data Files")
plt.ylabel("Average Value")
plt.grid(axis="y", linestyle="--", alpha=0.7)
for i, val in enumerate(averages):
    plt.text(i, val + 0.5, f"{val:.2f}", ha="center", fontsize=10)
plt.tight_layout()
plt.show()

# --- Plot per-day averages (optional trend view) ---
if per_day_avgs:
    days = list(per_day_avgs.keys())
    soc_vals = [per_day_avgs[d]["soc"] for d in days]
    grid_vals = [per_day_avgs[d]["grid"] for d in days]
    price_vals = [per_day_avgs[d]["price"] for d in days]
    cost_vals = [per_day_avgs[d]["cost"] for d in days]

    fig, axs = plt.subplots(4, 1, figsize=(10, 12), sharex=True)
    axs[0].plot(days, soc_vals, marker='o', color='blue')
    axs[0].set_ylabel("SoC (%)")
    axs[0].set_title("Average SoC per Day")

    axs[1].plot(days, grid_vals, marker='o', color='orange')
    axs[1].set_ylabel("Grid Usage")
    axs[1].set_title("Average Grid Usage per Day")

    axs[2].plot(days, price_vals, marker='o', color='green')
    axs[2].set_ylabel("Price")
    axs[2].set_title("Average Price per Day")

    axs[3].plot(days, cost_vals, marker='o', color='red')
    axs[3].set_ylabel("Cost")
    axs[3].set_title("Average Cost per Day")
    axs[3].set_xlabel("Day (file name)")

    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()
